# NFL Dataset Wrangling

## A. Pendahuluan
### A.1. **Pendahuluan**
Lorem ipsum

### A.2. **Alasan dataset menarik**
Lorem ipsum

### A.3. **Pertanyaan/tujuan analisis**
Lorem ipsum

In [ ]:
import pandas as pd



Gacoan Level 4.
